# TFM

## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import toml
import random
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

## Ejecución de código en GPU

In [2]:
import tensorflow as tf

# Comprobar la versión de TensorFlow
print("TensorFlow version:", tf.__version__)

# Listar todos los dispositivos físicos disponibles
physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print("🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):")
    for gpu in physical_devices:
        print(f'\t{gpu}')
else:
    print("🔴 No se ha detectado ninguna GPU. TensorFlow está utilizando la CPU.")

TensorFlow version: 2.10.0
🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):
	PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Lectura de fichero

In [3]:
data_dict = {}
DATA_DIR = '../../data/'

for data_folder in os.listdir(DATA_DIR):
    *data_files, config_file = os.listdir(f'{DATA_DIR}/{data_folder}')

    leak_value = toml.load(f'{DATA_DIR}/{data_folder}/{config_file}')['tanks']['flow_value']

    train_dataframe, test_dataframe = random.sample(data_files, 2)

    data_dict[data_folder] = {
        'leak_value': leak_value,
        'train_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{train_dataframe}').drop(columns=['Unnamed: 0']),
        'test_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{test_dataframe}').drop(columns=['Unnamed: 0'])
    }

### Conjuntos de datos de entrenamiento

In [4]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"20247,25",-5580,"6051,35","20718,6","20679,11","-39,49","-2,21",0
1,2,"20679,11",-6195,"7936,54","22420,65","22497,6","76,95","164,76",0
2,3,"22497,6",-6165,"5995,68","22328,28","22284,53","-43,75","73,79",0
3,4,"22284,53",-5720,"5031,76","21596,29","21654,39","58,1","181,92",0
4,5,"21654,39",-6075,"9124,56","24703,95","24903,47","199,52","323,61",0
...,...,...,...,...,...,...,...,...,...
995,996,"14567,32",-5990,"6995,54","15572,86","15611,86","39,0","-8,11",1
996,997,"15611,86",-6405,"6226,8","15433,66","15483,99","50,33","14,33",1
997,998,"15483,99",-5670,"5799,07","15613,06","15537,81","-75,25","6,19",1
998,999,"15537,81",-6130,"5099,01","14506,82","14515,34","8,52","84,27",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"12858,53","-6005,0","6028,71","12882,24","12839,43","-42,81","67,14",0
1,2,"12839,43","-5925,0","7060,68","13975,11","13920,05","-55,06","3,95",0
2,3,"13920,05","-6450,0","11910,17","19380,22","19446,63","66,41","-28,04",0
3,4,"19446,63","-6495,0","7059,14","20010,77","19951,08","-59,69","-92,41",0
4,5,"19951,08","-6300,0","7021,46","20672,54","20708,94","36,4","-130,33",0
...,...,...,...,...,...,...,...,...,...
995,996,"3854,5","-5960,0","5534,05","3428,55","3355,21","-73,34","-305,54",1
996,997,"3355,21","-6370,0","6329,75","3314,96","3330,31","15,35","-300,52",1
997,998,"3330,31","-5760,0","2905,72","476,03","483,45","7,42","-273,07",1
998,999,"483,45","-5412,11","8221,51","3292,85","3340,43","47,58","-195,76",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17445,49","-6140,0","8023,3","19328,79","19296,68","-32,11","150,41",0
1,2,"19296,68","-6010,0","5934,11","19220,79","19289,27","68,48","262,39",0
2,3,"19289,27","-5600,0","9215,39","22904,66","22910,69","6,03","227,65",0
3,4,"22910,69","-6070,0","6617,77","23458,46","23450,71","-7,75","172,64",0
4,5,"23450,71","-5685,0","3400,23","21165,94","21214,19","48,25","159,64",0
...,...,...,...,...,...,...,...,...,...
995,996,"0,72","-0,0","0,0","0,72","1,57","0,85","-3,08",1
996,997,"1,57","-0,0","0,0","1,57","11,07","9,5","1,17",1
997,998,"11,07","-0,0","0,0","11,07","6,2","-4,87","3,13",1
998,999,"6,2","-0,0","0,0","6,2","5,54","-0,66","-9,25",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17445,18","-5905,0","6012,94","17553,12","17533,44","-19,68","16,06",0
1,2,"17533,44","-6110,0","8506,7","19930,14","20021,08","90,94","102,33",0
2,3,"20021,08","-5930,0","4407,28","18498,36","18474,61","-23,75","162,19",0
3,4,"18474,61","-6080,0","8762,8","21157,41","21038,06","-119,35","-12,14",0
4,5,"21038,06","-6210,0","8428,75","23256,81","23192,92","-63,89","-100,6",0
...,...,...,...,...,...,...,...,...,...
995,996,"8,86","-0,0","0,0","8,86","9,4","0,54","3,9",1
996,997,"9,4","-0,0","0,0","9,4","12,79","3,39","2,6",1
997,998,"12,79","-0,0","0,0","12,79","14,69","1,9","-0,82",1
998,999,"14,69","-0,0","0,0","14,69","9,11","-5,58","-12,74",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21372,17","-5985,0","7788,9","23176,07","23214,64","38,57","19,95",0
1,2,"23214,64","-5765,0","2969,27","20418,91","20415,11","-3,8","61,14",0
2,3,"20415,11","-6325,0","7390,24","21480,35","21602,29","121,94","144,09",0
3,4,"21602,29","-6190,0","7315,88","22728,17","22691,65","-36,52","217,46",0
4,5,"22691,65","-5690,0","6031,24","23032,89","22983,03","-49,86","170,15",0
...,...,...,...,...,...,...,...,...,...
995,996,"17,19","-0,0","0,0","17,19","26,77","9,58","23,15",1
996,997,"26,77","-0,0","0,0","26,77","11,55","-15,22","10,12",1
997,998,"11,55","-0,0","0,0","11,55","1,45","-10,1","-1,74",1
998,999,"1,45","-0,0","0,0","1,45","8,2","6,75","5,73",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,65","-6055,0","6215,36","22304,01","22242,32","-61,69","34,11",0
1,2,"22242,32","-5940,0","6691,65","22993,97","22983,66","-10,31","42,41",0
2,3,"22983,66","-5885,0","6130,98","23229,64","23301,88","72,24","58,48",0
3,4,"23301,88","-6030,0","5035,77","22307,65","22270,88","-36,77","25,47",0
4,5,"22270,88","-6005,0","6518,15","22784,03","22794,42","10,39","13,96",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,74","-0,0","0,0","2,74","3,88","1,14","3,0",1
996,997,"3,88","-0,0","0,0","3,88","0,72","-3,16","-2,72",1
997,998,"0,72","-0,0","0,0","0,72","1,13","0,41","-11,02",1
998,999,"1,13","-0,0","0,0","1,13","2,14","1,01","-19,34",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"18109,75","-5940,0","6999,66","19169,41","19140,47","-28,94","290,5",0
1,2,"19140,47","-6320,0","7014,09","19834,56","19825,75","-8,81","346,46",0
2,3,"19825,75","-6145,0","7026,85","20707,6","20677,99","-29,61","326,59",0
3,4,"20677,99","-5805,0","4783,79","19656,78","19702,96","46,18","340,89",0
4,5,"19702,96","-6195,0","6592,28","20100,24","20021,69","-78,55","117,34",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,39","-0,0","0,0","3,39","8,94","5,55","-9,07",1
996,997,"8,94","-0,0","0,0","8,94","13,46","4,52","1,87",1
997,998,"13,46","-0,0","0,0","13,46","5,2","-8,26","4,92",1
998,999,"5,2","-0,0","0,0","5,2","13,18","7,98","-1,52",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22465,51","-6080,0","7789,55","24175,06","24208,87","33,81","-175,32",0
1,2,"24208,87","-6045,0","4948,93","23112,8","23139,14","26,34","-178,39",0
2,3,"23139,14","-5935,0","7036,17","24240,31","24285,57","45,26","-129,43",0
3,4,"24285,57","-5950,0","4812,44","23148,01","23215,86","67,85","74,7",0
4,5,"23215,86","-6310,0","5513,33","22419,19","22370,85","-48,34","67,2",0
...,...,...,...,...,...,...,...,...,...
995,996,"13,88","-0,0","0,0","13,88","5,95","-7,93","1,78",1
996,997,"5,95","-0,0","0,0","5,95","12,1","6,15","10,02",1
997,998,"12,1","-0,0","0,0","12,1","5,06","-7,04","-9,98",1
998,999,"5,06","-0,0","0,0","5,06","4,17","-0,89","-3,22",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17698,38","-5710,0","4618,64","16607,02","16580,51","-26,51","35,66",0
1,2,"16580,51","-5875,0","6004,3","16709,81","16702,31","-7,5","58,61",0
2,3,"16702,31","-6250,0","8001,28","18453,59","18456,14","2,55","-9,07",0
3,4,"18456,14","-5870,0","6931,83","19517,97","19595,08","77,11","-16,24",0
4,5,"19595,08","-6330,0","7449,42","20714,5","20644,03","-70,47","-108,89",0
...,...,...,...,...,...,...,...,...,...
995,996,"1,11","-0,0","0,0","1,11","1,53","0,42","-12,95",1
996,997,"1,53","-0,0","0,0","1,53","5,18","3,65","0,97",1
997,998,"5,18","-0,0","0,0","5,18","9,61","4,43","9,4",1
998,999,"9,61","-0,0","0,0","9,61","13,39","3,78","3,06",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16880,67","-6340,0","9520,82","20061,49","20089,2","27,71","239,98",0
1,2,"20089,2","-5965,0","8981,18","23105,38","23114,65","9,27","236,52",0
2,3,"23114,65","-6350,0","6284,23","23048,88","23080,01","31,13","96,76",0
3,4,"23080,01","-5495,0","3705,04","21290,05","21253,69","-36,36","173,15",0
4,5,"21253,69","-6010,0","6906,8","22150,49","22243,33","92,84","144,0",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,33","-0,0","0,0","2,33","8,66","6,33","4,22",1
996,997,"8,66","-0,0","0,0","8,66","8,69","0,03","3,5",1
997,998,"8,69","-0,0","0,0","8,69","21,02","12,33","20,09",1
998,999,"21,02","-0,0","0,0","21,02","5,64","-15,38","3,39",1


### Conjunto de datos de prueba

In [5]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16932,24",-5795,"9003,66","20140,9","20147,8","6,9","194,79",0
1,2,"20147,8",-5795,"8487,37","22840,17","22824,12","-16,05","51,05",0
2,3,"22824,12",-6285,"4549,79","21088,91","21147,56","58,65","78,96",0
3,4,"21147,56",-6085,"6664,97","21727,53","21782,08","54,55","88,64",0
4,5,"21782,08",-5950,"3939,69","19771,77","19772,17","0,4","188,2",0
...,...,...,...,...,...,...,...,...,...
995,996,"17751,65",-5940,"3360,0","15171,65","15219,44","47,79","-101,62",1
996,997,"15219,44",-6110,"4412,01","13521,45","13522,52","1,07","-153,6",1
997,998,"13522,52",-6235,"7069,05","14356,57","14257,47","-99,1","-289,6",1
998,999,"14257,47",-6190,"6181,85","14249,32","14242,17","-7,15","-280,19",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21149,68","-5960,0","5940,46","21130,14","21204,78","74,64","255,0",0
1,2,"21204,78","-6315,0","7021,73","21911,51","21862,93","-48,58","164,23",0
2,3,"21862,93","-5890,0","3218,08","19191,01","19194,42","3,41","149,26",0
3,4,"19194,42","-6160,0","6739,31","19773,73","19802,52","28,79","127,48",0
4,5,"19802,52","-5960,0","6979,79","20822,31","20853,99","31,68","148,24",0
...,...,...,...,...,...,...,...,...,...
995,996,"2228,36","-5465,58","4141,29","904,07","873,35","-30,72","-173,3",1
996,997,"873,35","-6090,0","6592,68","1376,03","1220,09","-155,94","-308,8",1
997,998,"1220,09","-5980,0","7181,83","2421,92","2463,95","42,03","-206,86",1
998,999,"2463,95","-5850,0","5758,71","2372,66","2400,72","28,06","-176,26",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,22","-5810,0","7001,2","23334,42","23347,02","12,6","-90,27",0
1,2,"23347,02","-6240,0","7270,93","24377,95","24283,54","-94,41","-105,27",0
2,3,"24283,54","-6430,0","3650,03","21503,57","21510,13","6,56","-17,7",0
3,4,"21510,13","-5850,0","6940,97","22601,1","22644,04","42,94","27,91",0
4,5,"22644,04","-6025,0","5910,84","22529,88","22565,92","36,04","80,35",0
...,...,...,...,...,...,...,...,...,...
995,996,"20,99","-0,0","0,0","20,99","7,38","-13,61","5,7",1
996,997,"7,38","-0,0","0,0","7,38","2,99","-4,39","-5,43",1
997,998,"2,99","-0,0","0,0","2,99","14,37","11,38","9,91",1
998,999,"14,37","-0,0","0,0","14,37","23,46","9,09","10,89",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"13689,29","-6200,0","9859,61","17348,9","17315,01","-33,89","93,19",0
1,2,"17315,01","-5965,0","6578,38","17928,39","18121,96","193,57","276,59",0
2,3,"18121,96","-6170,0","5339,44","17291,4","17357,0","65,6","264,32",0
3,4,"17357,0","-5975,0","6966,25","18348,25","18365,14","16,89","200,58",0
4,5,"18365,14","-6135,0","7964,66","20194,8","20222,38","27,58","192,82",0
...,...,...,...,...,...,...,...,...,...
995,996,"14,6","-0,0","0,0","14,6","3,95","-10,65","-1,28",1
996,997,"3,95","-0,0","0,0","3,95","8,56","4,61","-6,84",1
997,998,"8,56","-0,0","0,0","8,56","1,51","-7,05","-2,43",1
998,999,"1,51","-0,0","0,0","1,51","15,58","14,07","12,31",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"20535,61","-5940,0","8641,12","23236,73","23294,75","58,02","265,22",0
1,2,"23294,75","-6575,0","6455,26","23175,01","23296,05","121,04","387,15",0
2,3,"23296,05","-5630,0","6600,92","24266,97","24325,7","58,73","366,14",0
3,4,"24325,7","-6015,0","3190,31","21501,01","21523,46","22,45","375,09",0
4,5,"21523,46","-5695,0","7376,83","23205,29","23238,07","32,78","376,68",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,69","-0,0","0,0","3,69","23,8","20,11","18,39",1
996,997,"23,8","-0,0","0,0","23,8","15,45","-8,35","12,43",1
997,998,"15,45","-0,0","0,0","15,45","10,85","-4,6","-0,6",1
998,999,"10,85","-0,0","0,0","10,85","3,49","-7,36","-0,14",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21317,46","-5990,0","2968,12","18295,58","18323,64","28,06","-82,57",0
1,2,"18323,64","-6255,0","6219,19","18287,83","18284,2","-3,63","-120,57",0
2,3,"18284,2","-5830,0","5794,24","18248,44","18235,2","-13,24","18,69",0
3,4,"18235,2","-5855,0","6994,57","19374,77","19380,49","5,72","13,79",0
4,5,"19380,49","-6320,0","6978,26","20038,75","20051,59","12,84","35,39",0
...,...,...,...,...,...,...,...,...,...
995,996,"4,53","-0,0","0,0","4,53","4,38","-0,15","-0,18",1
996,997,"4,38","-0,0","0,0","4,38","21,36","16,98","10,55",1
997,998,"21,36","-0,0","0,0","21,36","15,47","-5,89","6,24",1
998,999,"15,47","-0,0","0,0","15,47","11,43","-4,04","11,26",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"15886,17","-5900,0","5970,3","15956,47","15980,01","23,54","68,46",0
1,2,"15980,01","-5810,0","6050,74","16220,75","16175,65","-45,1","8,64",0
2,3,"16175,65","-5760,0","5917,51","16333,16","16413,96","80,8","17,3",0
3,4,"16413,96","-5970,0","6982,8","17426,76","17460,02","33,26","68,23",0
4,5,"17460,02","-6340,0","7929,64","19049,66","19111,41","61,75","52,84",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,52","-0,0","0,0","5,52","0,47","-5,05","-4,84",1
996,997,"0,47","-0,0","0,0","0,47","5,79","5,32","0,68",1
997,998,"5,79","-0,0","0,0","5,79","14,14","8,35","-0,25",1
998,999,"14,14","-0,0","0,0","14,14","4,73","-9,41","2,66",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16777,0","-6100,0","9895,09","20572,09","20589,64","17,55","-18,03",0
1,2,"20589,64","-6115,0","4955,9","19430,54","19468,61","38,07","-12,02",0
2,3,"19468,61","-5965,0","6946,41","20450,02","20512,05","62,03","14,91",0
3,4,"20512,05","-6245,0","2980,24","17247,29","17265,48","18,19","87,81",0
4,5,"17265,48","-6425,0","8878,71","19719,19","19875,47","156,28","275,63",0
...,...,...,...,...,...,...,...,...,...
995,996,"0,33","-0,0","0,0","0,33","0,14","-0,19","-6,95",1
996,997,"0,14","-0,0","0,0","0,14","10,7","10,56","-1,35",1
997,998,"10,7","-0,0","0,0","10,7","11,2","0,5","4,17",1
998,999,"11,2","-0,0","0,0","11,2","11,98","0,78","-1,3",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"13606,56","-6135,0","6078,24","13549,8","13604,57","54,77","189,82",0
1,2,"13604,57","-6180,0","7816,29","15240,86","15298,64","57,78","140,18",0
2,3,"15298,64","-6540,0","8066,41","16825,05","16766,16","-58,89","114,81",0
3,4,"16766,16","-5950,0","2994,85","13811,01","13832,37","21,36","118,61",0
4,5,"13832,37","-6225,0","8046,4","15653,77","15600,11","-53,66","107,3",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,77","-0,0","0,0","3,77","5,54","1,77","-2,21",1
996,997,"5,54","-0,0","0,0","5,54","0,83","-4,71","-9,32",1
997,998,"0,83","-0,0","0,0","0,83","13,25","12,42","1,59",1
998,999,"13,25","-0,0","0,0","13,25","10,31","-2,94","6,54",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"11544,4","-5565,0","7069,47","13048,87","12982,38","-66,49","118,8",0
1,2,"12982,38","-6155,0","6953,56","13780,94","13840,48","59,54","124,42",0
2,3,"13840,48","-6005,0","8211,32","16046,8","15937,63","-109,17","-18,99",0
3,4,"15937,63","-5915,0","8550,06","18572,69","18595,78","23,09","76,99",0
4,5,"18595,78","-5830,0","6389,87","19155,65","19072,47","-83,18","-24,06",0
...,...,...,...,...,...,...,...,...,...
995,996,"16,15","-0,0","0,0","16,15","6,29","-9,86","-11,87",1
996,997,"6,29","-0,0","0,0","6,29","14,36","8,07","4,38",1
997,998,"14,36","-0,0","0,0","14,36","2,38","-11,98","-20,25",1
998,999,"2,38","-0,0","0,0","2,38","4,75","2,37","-6,32",1


## Limpieza de datos

In [6]:
bad_formatted_fields = [
    'Volumen dep. almacenam. ini. (L)',
    'Llenado dep. almacenam. (L)',
    'Volumen dep. almacenam. fin. teor. (L)',
    'Volumen dep. almacenam. fin. (L)',
    'Variacion',
    'Variacion Acum.',
]

for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        for field in bad_formatted_fields:
            data_dict[case][dataset][field] = data_dict[case][dataset][field].str.replace(',', '.')
            data_dict[case][dataset][field] = pd.to_numeric(data_dict[case][dataset][field])

### Conjunto de datos de entrenamiento

In [7]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,20247.25,-5580,6051.35,20718.60,20679.11,-39.49,-2.21,0
1,2,20679.11,-6195,7936.54,22420.65,22497.60,76.95,164.76,0
2,3,22497.60,-6165,5995.68,22328.28,22284.53,-43.75,73.79,0
3,4,22284.53,-5720,5031.76,21596.29,21654.39,58.10,181.92,0
4,5,21654.39,-6075,9124.56,24703.95,24903.47,199.52,323.61,0
...,...,...,...,...,...,...,...,...,...
995,996,14567.32,-5990,6995.54,15572.86,15611.86,39.00,-8.11,1
996,997,15611.86,-6405,6226.80,15433.66,15483.99,50.33,14.33,1
997,998,15483.99,-5670,5799.07,15613.06,15537.81,-75.25,6.19,1
998,999,15537.81,-6130,5099.01,14506.82,14515.34,8.52,84.27,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,12858.53,"-6005,0",6028.71,12882.24,12839.43,-42.81,67.14,0
1,2,12839.43,"-5925,0",7060.68,13975.11,13920.05,-55.06,3.95,0
2,3,13920.05,"-6450,0",11910.17,19380.22,19446.63,66.41,-28.04,0
3,4,19446.63,"-6495,0",7059.14,20010.77,19951.08,-59.69,-92.41,0
4,5,19951.08,"-6300,0",7021.46,20672.54,20708.94,36.40,-130.33,0
...,...,...,...,...,...,...,...,...,...
995,996,3854.50,"-5960,0",5534.05,3428.55,3355.21,-73.34,-305.54,1
996,997,3355.21,"-6370,0",6329.75,3314.96,3330.31,15.35,-300.52,1
997,998,3330.31,"-5760,0",2905.72,476.03,483.45,7.42,-273.07,1
998,999,483.45,"-5412,11",8221.51,3292.85,3340.43,47.58,-195.76,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17445.49,"-6140,0",8023.30,19328.79,19296.68,-32.11,150.41,0
1,2,19296.68,"-6010,0",5934.11,19220.79,19289.27,68.48,262.39,0
2,3,19289.27,"-5600,0",9215.39,22904.66,22910.69,6.03,227.65,0
3,4,22910.69,"-6070,0",6617.77,23458.46,23450.71,-7.75,172.64,0
4,5,23450.71,"-5685,0",3400.23,21165.94,21214.19,48.25,159.64,0
...,...,...,...,...,...,...,...,...,...
995,996,0.72,"-0,0",0.00,0.72,1.57,0.85,-3.08,1
996,997,1.57,"-0,0",0.00,1.57,11.07,9.50,1.17,1
997,998,11.07,"-0,0",0.00,11.07,6.20,-4.87,3.13,1
998,999,6.20,"-0,0",0.00,6.20,5.54,-0.66,-9.25,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17445.18,"-5905,0",6012.94,17553.12,17533.44,-19.68,16.06,0
1,2,17533.44,"-6110,0",8506.70,19930.14,20021.08,90.94,102.33,0
2,3,20021.08,"-5930,0",4407.28,18498.36,18474.61,-23.75,162.19,0
3,4,18474.61,"-6080,0",8762.80,21157.41,21038.06,-119.35,-12.14,0
4,5,21038.06,"-6210,0",8428.75,23256.81,23192.92,-63.89,-100.60,0
...,...,...,...,...,...,...,...,...,...
995,996,8.86,"-0,0",0.00,8.86,9.40,0.54,3.90,1
996,997,9.40,"-0,0",0.00,9.40,12.79,3.39,2.60,1
997,998,12.79,"-0,0",0.00,12.79,14.69,1.90,-0.82,1
998,999,14.69,"-0,0",0.00,14.69,9.11,-5.58,-12.74,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21372.17,"-5985,0",7788.90,23176.07,23214.64,38.57,19.95,0
1,2,23214.64,"-5765,0",2969.27,20418.91,20415.11,-3.80,61.14,0
2,3,20415.11,"-6325,0",7390.24,21480.35,21602.29,121.94,144.09,0
3,4,21602.29,"-6190,0",7315.88,22728.17,22691.65,-36.52,217.46,0
4,5,22691.65,"-5690,0",6031.24,23032.89,22983.03,-49.86,170.15,0
...,...,...,...,...,...,...,...,...,...
995,996,17.19,"-0,0",0.00,17.19,26.77,9.58,23.15,1
996,997,26.77,"-0,0",0.00,26.77,11.55,-15.22,10.12,1
997,998,11.55,"-0,0",0.00,11.55,1.45,-10.10,-1.74,1
998,999,1.45,"-0,0",0.00,1.45,8.20,6.75,5.73,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.65,"-6055,0",6215.36,22304.01,22242.32,-61.69,34.11,0
1,2,22242.32,"-5940,0",6691.65,22993.97,22983.66,-10.31,42.41,0
2,3,22983.66,"-5885,0",6130.98,23229.64,23301.88,72.24,58.48,0
3,4,23301.88,"-6030,0",5035.77,22307.65,22270.88,-36.77,25.47,0
4,5,22270.88,"-6005,0",6518.15,22784.03,22794.42,10.39,13.96,0
...,...,...,...,...,...,...,...,...,...
995,996,2.74,"-0,0",0.00,2.74,3.88,1.14,3.00,1
996,997,3.88,"-0,0",0.00,3.88,0.72,-3.16,-2.72,1
997,998,0.72,"-0,0",0.00,0.72,1.13,0.41,-11.02,1
998,999,1.13,"-0,0",0.00,1.13,2.14,1.01,-19.34,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,18109.75,"-5940,0",6999.66,19169.41,19140.47,-28.94,290.50,0
1,2,19140.47,"-6320,0",7014.09,19834.56,19825.75,-8.81,346.46,0
2,3,19825.75,"-6145,0",7026.85,20707.60,20677.99,-29.61,326.59,0
3,4,20677.99,"-5805,0",4783.79,19656.78,19702.96,46.18,340.89,0
4,5,19702.96,"-6195,0",6592.28,20100.24,20021.69,-78.55,117.34,0
...,...,...,...,...,...,...,...,...,...
995,996,3.39,"-0,0",0.00,3.39,8.94,5.55,-9.07,1
996,997,8.94,"-0,0",0.00,8.94,13.46,4.52,1.87,1
997,998,13.46,"-0,0",0.00,13.46,5.20,-8.26,4.92,1
998,999,5.20,"-0,0",0.00,5.20,13.18,7.98,-1.52,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22465.51,"-6080,0",7789.55,24175.06,24208.87,33.81,-175.32,0
1,2,24208.87,"-6045,0",4948.93,23112.80,23139.14,26.34,-178.39,0
2,3,23139.14,"-5935,0",7036.17,24240.31,24285.57,45.26,-129.43,0
3,4,24285.57,"-5950,0",4812.44,23148.01,23215.86,67.85,74.70,0
4,5,23215.86,"-6310,0",5513.33,22419.19,22370.85,-48.34,67.20,0
...,...,...,...,...,...,...,...,...,...
995,996,13.88,"-0,0",0.00,13.88,5.95,-7.93,1.78,1
996,997,5.95,"-0,0",0.00,5.95,12.10,6.15,10.02,1
997,998,12.10,"-0,0",0.00,12.10,5.06,-7.04,-9.98,1
998,999,5.06,"-0,0",0.00,5.06,4.17,-0.89,-3.22,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17698.38,"-5710,0",4618.64,16607.02,16580.51,-26.51,35.66,0
1,2,16580.51,"-5875,0",6004.30,16709.81,16702.31,-7.50,58.61,0
2,3,16702.31,"-6250,0",8001.28,18453.59,18456.14,2.55,-9.07,0
3,4,18456.14,"-5870,0",6931.83,19517.97,19595.08,77.11,-16.24,0
4,5,19595.08,"-6330,0",7449.42,20714.50,20644.03,-70.47,-108.89,0
...,...,...,...,...,...,...,...,...,...
995,996,1.11,"-0,0",0.00,1.11,1.53,0.42,-12.95,1
996,997,1.53,"-0,0",0.00,1.53,5.18,3.65,0.97,1
997,998,5.18,"-0,0",0.00,5.18,9.61,4.43,9.40,1
998,999,9.61,"-0,0",0.00,9.61,13.39,3.78,3.06,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16880.67,"-6340,0",9520.82,20061.49,20089.20,27.71,239.98,0
1,2,20089.20,"-5965,0",8981.18,23105.38,23114.65,9.27,236.52,0
2,3,23114.65,"-6350,0",6284.23,23048.88,23080.01,31.13,96.76,0
3,4,23080.01,"-5495,0",3705.04,21290.05,21253.69,-36.36,173.15,0
4,5,21253.69,"-6010,0",6906.80,22150.49,22243.33,92.84,144.00,0
...,...,...,...,...,...,...,...,...,...
995,996,2.33,"-0,0",0.00,2.33,8.66,6.33,4.22,1
996,997,8.66,"-0,0",0.00,8.66,8.69,0.03,3.50,1
997,998,8.69,"-0,0",0.00,8.69,21.02,12.33,20.09,1
998,999,21.02,"-0,0",0.00,21.02,5.64,-15.38,3.39,1


### Conjunto de datos de prueba

In [8]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16932.24,-5795,9003.66,20140.90,20147.80,6.90,194.79,0
1,2,20147.80,-5795,8487.37,22840.17,22824.12,-16.05,51.05,0
2,3,22824.12,-6285,4549.79,21088.91,21147.56,58.65,78.96,0
3,4,21147.56,-6085,6664.97,21727.53,21782.08,54.55,88.64,0
4,5,21782.08,-5950,3939.69,19771.77,19772.17,0.40,188.20,0
...,...,...,...,...,...,...,...,...,...
995,996,17751.65,-5940,3360.00,15171.65,15219.44,47.79,-101.62,1
996,997,15219.44,-6110,4412.01,13521.45,13522.52,1.07,-153.60,1
997,998,13522.52,-6235,7069.05,14356.57,14257.47,-99.10,-289.60,1
998,999,14257.47,-6190,6181.85,14249.32,14242.17,-7.15,-280.19,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21149.68,"-5960,0",5940.46,21130.14,21204.78,74.64,255.00,0
1,2,21204.78,"-6315,0",7021.73,21911.51,21862.93,-48.58,164.23,0
2,3,21862.93,"-5890,0",3218.08,19191.01,19194.42,3.41,149.26,0
3,4,19194.42,"-6160,0",6739.31,19773.73,19802.52,28.79,127.48,0
4,5,19802.52,"-5960,0",6979.79,20822.31,20853.99,31.68,148.24,0
...,...,...,...,...,...,...,...,...,...
995,996,2228.36,"-5465,58",4141.29,904.07,873.35,-30.72,-173.30,1
996,997,873.35,"-6090,0",6592.68,1376.03,1220.09,-155.94,-308.80,1
997,998,1220.09,"-5980,0",7181.83,2421.92,2463.95,42.03,-206.86,1
998,999,2463.95,"-5850,0",5758.71,2372.66,2400.72,28.06,-176.26,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.22,"-5810,0",7001.20,23334.42,23347.02,12.60,-90.27,0
1,2,23347.02,"-6240,0",7270.93,24377.95,24283.54,-94.41,-105.27,0
2,3,24283.54,"-6430,0",3650.03,21503.57,21510.13,6.56,-17.70,0
3,4,21510.13,"-5850,0",6940.97,22601.10,22644.04,42.94,27.91,0
4,5,22644.04,"-6025,0",5910.84,22529.88,22565.92,36.04,80.35,0
...,...,...,...,...,...,...,...,...,...
995,996,20.99,"-0,0",0.00,20.99,7.38,-13.61,5.70,1
996,997,7.38,"-0,0",0.00,7.38,2.99,-4.39,-5.43,1
997,998,2.99,"-0,0",0.00,2.99,14.37,11.38,9.91,1
998,999,14.37,"-0,0",0.00,14.37,23.46,9.09,10.89,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,13689.29,"-6200,0",9859.61,17348.90,17315.01,-33.89,93.19,0
1,2,17315.01,"-5965,0",6578.38,17928.39,18121.96,193.57,276.59,0
2,3,18121.96,"-6170,0",5339.44,17291.40,17357.00,65.60,264.32,0
3,4,17357.00,"-5975,0",6966.25,18348.25,18365.14,16.89,200.58,0
4,5,18365.14,"-6135,0",7964.66,20194.80,20222.38,27.58,192.82,0
...,...,...,...,...,...,...,...,...,...
995,996,14.60,"-0,0",0.00,14.60,3.95,-10.65,-1.28,1
996,997,3.95,"-0,0",0.00,3.95,8.56,4.61,-6.84,1
997,998,8.56,"-0,0",0.00,8.56,1.51,-7.05,-2.43,1
998,999,1.51,"-0,0",0.00,1.51,15.58,14.07,12.31,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,20535.61,"-5940,0",8641.12,23236.73,23294.75,58.02,265.22,0
1,2,23294.75,"-6575,0",6455.26,23175.01,23296.05,121.04,387.15,0
2,3,23296.05,"-5630,0",6600.92,24266.97,24325.70,58.73,366.14,0
3,4,24325.70,"-6015,0",3190.31,21501.01,21523.46,22.45,375.09,0
4,5,21523.46,"-5695,0",7376.83,23205.29,23238.07,32.78,376.68,0
...,...,...,...,...,...,...,...,...,...
995,996,3.69,"-0,0",0.00,3.69,23.80,20.11,18.39,1
996,997,23.80,"-0,0",0.00,23.80,15.45,-8.35,12.43,1
997,998,15.45,"-0,0",0.00,15.45,10.85,-4.60,-0.60,1
998,999,10.85,"-0,0",0.00,10.85,3.49,-7.36,-0.14,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21317.46,"-5990,0",2968.12,18295.58,18323.64,28.06,-82.57,0
1,2,18323.64,"-6255,0",6219.19,18287.83,18284.20,-3.63,-120.57,0
2,3,18284.20,"-5830,0",5794.24,18248.44,18235.20,-13.24,18.69,0
3,4,18235.20,"-5855,0",6994.57,19374.77,19380.49,5.72,13.79,0
4,5,19380.49,"-6320,0",6978.26,20038.75,20051.59,12.84,35.39,0
...,...,...,...,...,...,...,...,...,...
995,996,4.53,"-0,0",0.00,4.53,4.38,-0.15,-0.18,1
996,997,4.38,"-0,0",0.00,4.38,21.36,16.98,10.55,1
997,998,21.36,"-0,0",0.00,21.36,15.47,-5.89,6.24,1
998,999,15.47,"-0,0",0.00,15.47,11.43,-4.04,11.26,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,15886.17,"-5900,0",5970.30,15956.47,15980.01,23.54,68.46,0
1,2,15980.01,"-5810,0",6050.74,16220.75,16175.65,-45.10,8.64,0
2,3,16175.65,"-5760,0",5917.51,16333.16,16413.96,80.80,17.30,0
3,4,16413.96,"-5970,0",6982.80,17426.76,17460.02,33.26,68.23,0
4,5,17460.02,"-6340,0",7929.64,19049.66,19111.41,61.75,52.84,0
...,...,...,...,...,...,...,...,...,...
995,996,5.52,"-0,0",0.00,5.52,0.47,-5.05,-4.84,1
996,997,0.47,"-0,0",0.00,0.47,5.79,5.32,0.68,1
997,998,5.79,"-0,0",0.00,5.79,14.14,8.35,-0.25,1
998,999,14.14,"-0,0",0.00,14.14,4.73,-9.41,2.66,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16777.00,"-6100,0",9895.09,20572.09,20589.64,17.55,-18.03,0
1,2,20589.64,"-6115,0",4955.90,19430.54,19468.61,38.07,-12.02,0
2,3,19468.61,"-5965,0",6946.41,20450.02,20512.05,62.03,14.91,0
3,4,20512.05,"-6245,0",2980.24,17247.29,17265.48,18.19,87.81,0
4,5,17265.48,"-6425,0",8878.71,19719.19,19875.47,156.28,275.63,0
...,...,...,...,...,...,...,...,...,...
995,996,0.33,"-0,0",0.00,0.33,0.14,-0.19,-6.95,1
996,997,0.14,"-0,0",0.00,0.14,10.70,10.56,-1.35,1
997,998,10.70,"-0,0",0.00,10.70,11.20,0.50,4.17,1
998,999,11.20,"-0,0",0.00,11.20,11.98,0.78,-1.30,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,13606.56,"-6135,0",6078.24,13549.80,13604.57,54.77,189.82,0
1,2,13604.57,"-6180,0",7816.29,15240.86,15298.64,57.78,140.18,0
2,3,15298.64,"-6540,0",8066.41,16825.05,16766.16,-58.89,114.81,0
3,4,16766.16,"-5950,0",2994.85,13811.01,13832.37,21.36,118.61,0
4,5,13832.37,"-6225,0",8046.40,15653.77,15600.11,-53.66,107.30,0
...,...,...,...,...,...,...,...,...,...
995,996,3.77,"-0,0",0.00,3.77,5.54,1.77,-2.21,1
996,997,5.54,"-0,0",0.00,5.54,0.83,-4.71,-9.32,1
997,998,0.83,"-0,0",0.00,0.83,13.25,12.42,1.59,1
998,999,13.25,"-0,0",0.00,13.25,10.31,-2.94,6.54,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,11544.40,"-5565,0",7069.47,13048.87,12982.38,-66.49,118.80,0
1,2,12982.38,"-6155,0",6953.56,13780.94,13840.48,59.54,124.42,0
2,3,13840.48,"-6005,0",8211.32,16046.80,15937.63,-109.17,-18.99,0
3,4,15937.63,"-5915,0",8550.06,18572.69,18595.78,23.09,76.99,0
4,5,18595.78,"-5830,0",6389.87,19155.65,19072.47,-83.18,-24.06,0
...,...,...,...,...,...,...,...,...,...
995,996,16.15,"-0,0",0.00,16.15,6.29,-9.86,-11.87,1
996,997,6.29,"-0,0",0.00,6.29,14.36,8.07,4.38,1
997,998,14.36,"-0,0",0.00,14.36,2.38,-11.98,-20.25,1
998,999,2.38,"-0,0",0.00,2.38,4.75,2.37,-6.32,1


<!-- TODO -->

## Selección de características

In [9]:
for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        data_dict[case][dataset] = data_dict[case][dataset].filter(items=['Variacion', 'Fugando combustible'])

### Conjunto de datos de entrenamiento

In [10]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Variacion,Fugando combustible
0,-39.49,0
1,76.95,0
2,-43.75,0
3,58.10,0
4,199.52,0
...,...,...
995,39.00,1
996,50.33,1
997,-75.25,1
998,8.52,1


,Variacion,Fugando combustible
0,-42.81,0
1,-55.06,0
2,66.41,0
3,-59.69,0
4,36.40,0
...,...,...
995,-73.34,1
996,15.35,1
997,7.42,1
998,47.58,1


,Variacion,Fugando combustible
0,-32.11,0
1,68.48,0
2,6.03,0
3,-7.75,0
4,48.25,0
...,...,...
995,0.85,1
996,9.50,1
997,-4.87,1
998,-0.66,1


,Variacion,Fugando combustible
0,-19.68,0
1,90.94,0
2,-23.75,0
3,-119.35,0
4,-63.89,0
...,...,...
995,0.54,1
996,3.39,1
997,1.90,1
998,-5.58,1


,Variacion,Fugando combustible
0,38.57,0
1,-3.80,0
2,121.94,0
3,-36.52,0
4,-49.86,0
...,...,...
995,9.58,1
996,-15.22,1
997,-10.10,1
998,6.75,1


...

,Variacion,Fugando combustible
0,-61.69,0
1,-10.31,0
2,72.24,0
3,-36.77,0
4,10.39,0
...,...,...
995,1.14,1
996,-3.16,1
997,0.41,1
998,1.01,1


,Variacion,Fugando combustible
0,-28.94,0
1,-8.81,0
2,-29.61,0
3,46.18,0
4,-78.55,0
...,...,...
995,5.55,1
996,4.52,1
997,-8.26,1
998,7.98,1


,Variacion,Fugando combustible
0,33.81,0
1,26.34,0
2,45.26,0
3,67.85,0
4,-48.34,0
...,...,...
995,-7.93,1
996,6.15,1
997,-7.04,1
998,-0.89,1


,Variacion,Fugando combustible
0,-26.51,0
1,-7.50,0
2,2.55,0
3,77.11,0
4,-70.47,0
...,...,...
995,0.42,1
996,3.65,1
997,4.43,1
998,3.78,1


,Variacion,Fugando combustible
0,27.71,0
1,9.27,0
2,31.13,0
3,-36.36,0
4,92.84,0
...,...,...
995,6.33,1
996,0.03,1
997,12.33,1
998,-15.38,1


### Conjunto de datos de prueba

In [11]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Variacion,Fugando combustible
0,6.90,0
1,-16.05,0
2,58.65,0
3,54.55,0
4,0.40,0
...,...,...
995,47.79,1
996,1.07,1
997,-99.10,1
998,-7.15,1


,Variacion,Fugando combustible
0,74.64,0
1,-48.58,0
2,3.41,0
3,28.79,0
4,31.68,0
...,...,...
995,-30.72,1
996,-155.94,1
997,42.03,1
998,28.06,1


,Variacion,Fugando combustible
0,12.60,0
1,-94.41,0
2,6.56,0
3,42.94,0
4,36.04,0
...,...,...
995,-13.61,1
996,-4.39,1
997,11.38,1
998,9.09,1


,Variacion,Fugando combustible
0,-33.89,0
1,193.57,0
2,65.60,0
3,16.89,0
4,27.58,0
...,...,...
995,-10.65,1
996,4.61,1
997,-7.05,1
998,14.07,1


,Variacion,Fugando combustible
0,58.02,0
1,121.04,0
2,58.73,0
3,22.45,0
4,32.78,0
...,...,...
995,20.11,1
996,-8.35,1
997,-4.60,1
998,-7.36,1


...

,Variacion,Fugando combustible
0,28.06,0
1,-3.63,0
2,-13.24,0
3,5.72,0
4,12.84,0
...,...,...
995,-0.15,1
996,16.98,1
997,-5.89,1
998,-4.04,1


,Variacion,Fugando combustible
0,23.54,0
1,-45.10,0
2,80.80,0
3,33.26,0
4,61.75,0
...,...,...
995,-5.05,1
996,5.32,1
997,8.35,1
998,-9.41,1


,Variacion,Fugando combustible
0,17.55,0
1,38.07,0
2,62.03,0
3,18.19,0
4,156.28,0
...,...,...
995,-0.19,1
996,10.56,1
997,0.50,1
998,0.78,1


,Variacion,Fugando combustible
0,54.77,0
1,57.78,0
2,-58.89,0
3,21.36,0
4,-53.66,0
...,...,...
995,1.77,1
996,-4.71,1
997,12.42,1
998,-2.94,1


,Variacion,Fugando combustible
0,-66.49,0
1,59.54,0
2,-109.17,0
3,23.09,0
4,-83.18,0
...,...,...
995,-9.86,1
996,8.07,1
997,-11.98,1
998,2.37,1


<!-- TODO -->

## Unificación de conjunto de datos

In [12]:
complete_train_dataframe = pd.concat([data_dict[key]['train_dataframe'] for key in data_dict])

X_train_complete = complete_train_dataframe.drop(columns='Fugando combustible')
y_train_complete = complete_train_dataframe['Fugando combustible']

## Implementación de modelos

### Definición de métricas

In [13]:
def leak_detection_delay(y_true: np.array, y_pred: np.array) -> int:
    real_first_leak_day = 0
    while y_true[real_first_leak_day] != 1: real_first_leak_day += 1

    pred_first_leak_day = 0
    while y_pred[pred_first_leak_day] != 1: pred_first_leak_day += 1

    return (pred_first_leak_day - real_first_leak_day)

### Aplicación de `GridSearch` a modelos

In [14]:
from sklearn.model_selection import GridSearchCV

def eval_classic_model(base_classifier, param_grid, cv = 5) -> None:
    grid_search = GridSearchCV(
        estimator = base_classifier,
        param_grid = param_grid,
        cv = cv
    )

    grid_search.fit(X_train_complete, y_train_complete)

    print(f"Best parameters found: {grid_search.best_params_}")

    result_dict = {}

    best_classifier = grid_search.best_estimator_
    for case in data_dict:
        leak_value = data_dict[case]['leak_value']
        test_dataframe = data_dict[case]['test_dataframe']

        X_test = test_dataframe.drop(columns='Fugando combustible')
        y_test = test_dataframe['Fugando combustible']

        best_classifier = grid_search.best_estimator_
        y_pred = best_classifier.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        leak_delay = leak_detection_delay(y_test, y_pred)

        result_dict[leak_value] = [accuracy, leak_delay]
    
    display(pd.DataFrame.from_dict(
        data = result_dict,
        orient = 'index',
        columns = ['accuracy_score', 'leak_detection_delay']
    ))

### Modelos clásicos

#### Random Forest Classifier

In [15]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier()

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

eval_classic_model(rf_classifier, rf_param_grid)

c:\Users\cabre\anaconda3\envs\py310\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\cabre\anaconda3\envs\py310\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\cabre\anaconda3\envs\py310\lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\cabre\anaconda3\envs\py310\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  F

Best parameters found: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}


,accuracy_score,leak_detection_delay
0.013000,0.529,-495
0.054929,0.525,-493
0.096857,0.662,-495
0.138786,0.703,-490
0.180714,0.761,-489
0.222643,0.779,-486
0.264571,0.797,-494
0.306500,0.808,-492
0.348429,0.826,-492
0.390357,0.808,-493


#### Naive Bayes Classifier

In [16]:
from sklearn.naive_bayes import GaussianNB

gnb_classifier = GaussianNB()

gnb_param_grid = {
    'var_smoothing': [1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

eval_classic_model(gnb_classifier, gnb_param_grid)

Best parameters found: {'var_smoothing': 1e-11}


,accuracy_score,leak_detection_delay
0.013000,0.525,-485
0.054929,0.608,-494
0.096857,0.607,-494
0.138786,0.599,-494
0.180714,0.555,-494
0.222643,0.548,-486
0.264571,0.524,-489
0.306500,0.517,-493
0.348429,0.501,-495
0.390357,0.508,-491


#### KNN Classifier

In [17]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier()

knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

eval_classic_model(knn_classifier, knn_param_grid)

Best parameters found: {'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'uniform'}


,accuracy_score,leak_detection_delay
0.013000,0.534,-495
0.054929,0.534,-493
0.096857,0.650,-495
0.138786,0.706,-495
0.180714,0.751,-489
0.222643,0.765,-486
0.264571,0.774,-495
0.306500,0.791,-492
0.348429,0.820,-492
0.390357,0.792,-491


#### XGBoost Classifier

In [18]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier()

xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

eval_classic_model(xgb_classifier, xgb_param_grid)

Best parameters found: {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1.0}


,accuracy_score,leak_detection_delay
0.013000,0.537,-495
0.054929,0.520,-493
0.096857,0.661,-495
0.138786,0.698,-490
0.180714,0.760,-489
0.222643,0.788,-486
0.264571,0.799,-494
0.306500,0.807,-487
0.348429,0.831,-488
0.390357,0.819,-493


### Múltiples modelos de _Deep Learning_

In [19]:
result_dict = {}
for case in data_dict:
    X_train = data_dict[case]['train_dataframe'].drop(columns='Fugando combustible')
    y_train = data_dict[case]['train_dataframe']['Fugando combustible']
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
        tf.keras.layers.LSTM(15, dropout=0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = 50,
        batch_size = 8192,
        verbose = 0
    )

    leak_value = data_dict[case]['leak_value']

    X_test = data_dict[case]['test_dataframe'].drop(columns='Fugando combustible')
    y_test = data_dict[case]['test_dataframe']['Fugando combustible']
    
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

    test_accuracy = accuracy_score(y_test, y_pred)

    test_leak_delay = leak_detection_delay(y_test, y_pred)

    result_dict[leak_value] = (test_accuracy, test_leak_delay)

display(pd.DataFrame.from_dict(
    data = result_dict,
    orient = 'index',
    columns = ['accuracy_score', 'leak_detection_delay']
))

,accuracy_score,leak_detection_delay
0.013000,0.570,-495
0.054929,0.513,-495
0.096857,0.699,-494
0.138786,0.680,-495
0.180714,0.743,-489
0.222643,0.625,-492
0.264571,0.598,-495
0.306500,0.647,-491
0.348429,0.393,-495
0.390357,0.816,-493


### Único modelo

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_complete.shape[1], 1)),
    tf.keras.layers.LSTM(15, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x = X_train_complete,
    y = y_train_complete,
    epochs = 50,
    batch_size = 8192,
    verbose = 0
)

result_dict = {}
for case in data_dict:
    leak_value = data_dict[case]['leak_value']
    X_test = data_dict[case]['test_dataframe'].drop(columns='Fugando combustible')
    y_test = data_dict[case]['test_dataframe']['Fugando combustible']
    
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

    test_accuracy = accuracy_score(y_test, y_pred)

    test_leak_delay = leak_detection_delay(y_test, y_pred)

    result_dict[leak_value] = (test_accuracy, test_leak_delay)

display(pd.DataFrame.from_dict(
    data = result_dict,
    orient = 'index',
    columns = ['accuracy_score', 'leak_detection_delay']
))

,accuracy_score,leak_detection_delay
0.013000,0.515,-495
0.054929,0.477,-493
0.096857,0.568,-495
0.138786,0.614,-490
0.180714,0.675,-489
0.222643,0.685,-489
0.264571,0.711,-494
0.306500,0.727,-491
0.348429,0.746,-488
0.390357,0.746,-493
